In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [5]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
# using StaticUnivariatePolynomials
using Polynomials
using BenchmarkTools
using TransportMap

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


### Hermite polynomials

In [13]:
@btime phyhermite_coeffmatrix(5)

  621.205 ns (21 allocations: 2.23 KiB)


6×6 Array{Float64,2}:
  1.0    0.0    0.0     0.0   0.0   0.0
  0.0    2.0    0.0     0.0   0.0   0.0
 -2.0    0.0    4.0     0.0   0.0   0.0
 -0.0  -12.0    0.0     8.0   0.0   0.0
 12.0   -0.0  -48.0     0.0  16.0   0.0
  0.0  120.0   -0.0  -160.0   0.0  32.0

In [15]:
for i=0:10
    P = PhyHermite(i;scaled=true)
    @show P
end

P = PhyHermite{0}(ImmutablePolynomial(1.0), ImmutablePolynomial(0.0), ImmutablePolynomial(0.0), true)
P = PhyHermite{1}(ImmutablePolynomial(1.0622519320271968*x), ImmutablePolynomial(1.0622519320271968), ImmutablePolynomial(0.0), true)
P = PhyHermite{2}(ImmutablePolynomial(-0.5311259660135984 + 1.0622519320271968*x^2), ImmutablePolynomial(2.1245038640543936*x), ImmutablePolynomial(1.0622519320271968), true)
P = PhyHermite{3}(ImmutablePolynomial(-1.3009876058761163*x + 0.8673250705840775*x^3), ImmutablePolynomial(-1.3009876058761163 + 2.6019752117522326*x^2), ImmutablePolynomial(2.6019752117522326*x), true)
P = PhyHermite{4}(ImmutablePolynomial(0.4599685791773267 - 1.8398743167093068*x^2 + 0.6132914389031022*x^4), ImmutablePolynomial(-3.6797486334186136*x + 2.453165755612409*x^3), ImmutablePolynomial(-1.8398743167093068 + 3.6797486334186136*x^2), true)
P = PhyHermite{5}(ImmutablePolynomial(1.4545483623118505*x - 1.9393978164158008*x^3 + 0.38787956328316014*x^5), ImmutablePolynomial(1.45

In [5]:
typeof(ImmutablePolynomial)<:Polynomial

false

In [18]:
@btime P = PhyHermite(10)

  3.353 μs (28 allocations: 2.14 KiB)


PhyHermite{10}(ImmutablePolynomial(-30240.0 + 302400.0*x^2 - 403200.0*x^4 + 161280.0*x^6 - 23040.0*x^8 + 1024.0*x^10), ImmutablePolynomial(604800.0*x - 1.6128e6*x^3 + 967680.0*x^5 - 184320.0*x^7 + 10240.0*x^9), ImmutablePolynomial(302400.0 - 2.4192e6*x^2 + 2.4192e6*x^4 - 645120.0*x^6 + 46080.0*x^8), false)

In [ ]:
@time derivative(P)

In [21]:
m = 10
@btime 2*m*(m-1)*view(PhyH,m-1,1:m-1)

  118.368 ns (3 allocations: 256 bytes)


9-element Array{Float64,1}:
  302400.0
      -0.0
      -2.4192e6
       0.0
       2.4192e6
      -0.0
 -645120.0
       0.0
   46080.0

In [22]:
@btime 2*m*(m-1)*PhyH[m-1,1:m-1]

  139.096 ns (3 allocations: 352 bytes)


9-element Array{Float64,1}:
  302400.0
      -0.0
      -2.4192e6
       0.0
       2.4192e6
      -0.0
 -645120.0
       0.0
   46080.0

In [22]:
derivative(P.P, 1.0)

MethodError: MethodError: no method matching derivative(::ImmutablePolynomial{Float64,11}, ::Float64)
Closest candidates are:
  derivative(::P) where {T, P<:Polynomials.StandardBasisPolynomial{T}} at /home/mat/.julia/packages/Polynomials/pF6Ck/src/polynomials/standard-basis.jl:50
  derivative(::P, !Matched::Integer) where {T, P<:Polynomials.StandardBasisPolynomial{T}} at /home/mat/.julia/packages/Polynomials/pF6Ck/src/polynomials/standard-basis.jl:50

In [20]:
@btime P.P*P.P

  75.657 ns (1 allocation: 192 bytes)


ImmutablePolynomial(9.144576e8 - 1.8289152e10*x^2 + 1.15831296e11*x^4 - 2.536095744e11*x^6 + 2.615058432e11*x^8 - 1.4405271552e11*x^10 + 4.52100096e10*x^12 - 8.257536e9*x^14 + 8.6114304e8*x^16 - 4.718592e7*x^18 + 1.048576e6*x^20)

In [21]:
P(1.0)*P(1.0)-(P.P*P.P)(1.0)

0.0

In [17]:
function evalpoly()
    @btime P.P(1.0)
end

evalpoly (generic function with 1 method)

In [18]:
evalpoly()

  31.744 ns (1 allocation: 16 bytes)


8224.0

In [7]:
@btime derivative(P)

MethodError: MethodError: no method matching derivative(::PhyHermite{20})
Closest candidates are:
  derivative(!Matched::SparsePolynomial{T}) where T at /home/mat/.julia/packages/Polynomials/pF6Ck/src/polynomials/SparsePolynomial.jl:270
  derivative(!Matched::SparsePolynomial{T}, !Matched::Integer) where T at /home/mat/.julia/packages/Polynomials/pF6Ck/src/polynomials/SparsePolynomial.jl:270
  derivative(!Matched::P) where {T, P<:LaurentPolynomial{T}} at /home/mat/.julia/packages/Polynomials/pF6Ck/src/polynomials/LaurentPolynomial.jl:373
  ...

In [24]:
differentiate(P)

UndefVarError: UndefVarError: differentiate not defined

In [21]:
typeof(P)

ImmutablePolynomial{Float64,11}

In [23]:
?vander

search: vander



```
vander(::Type{AbstractPolynomial}, x::AbstractVector, deg::Integer)
```

Calculate the psuedo-Vandermonde matrix of the given polynomial type with the given degree.

# References

[Vandermonde Matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix)


In [30]:
@btime P(1.0)

  19.135 ns (1 allocation: 16 bytes)


1.107214478336e12

In [37]:
@btime randn()

  5.811 ns (0 allocations: 0 bytes)


-1.4670797765696044

In [34]:
@btime P20($randn())

  20.327 ns (0 allocations: 0 bytes)


4.753304645718954e11

In [20]:
P = phyhermite(0)

┌ Error: Failed to revise /media/mat/HDD/AdaptiveTransportMap/src/phyhermite.jl
│   exception = (ErrorException("lowering returned an error, \$(Expr(:error, \"\\\"0\\\" is not a valid function argument name\"))"), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007f1f51576d6c, Ptr{Nothing} @0x00007f1f5157717f])
└ @ Revise /home/mat/.julia/packages/Revise/BqeJF/src/Revise.jl:760
┌ Warning: Due to a previously reported error, the running code does not match saved version for the following files:
│ 
│   /media/mat/HDD/AdaptiveTransportMap/src/phyhermite.jl
│ 
│ Use Revise.errors() to report errors again.
└ @ Revise /home/mat/.julia/packages/Revise/BqeJF/src/Revise.jl:820


BoundsError: BoundsError: attempt to access 21×21 Array{Float64,2} at index [0, 1:0]

In [18]:
P(2.0)

1.0

In [37]:
@time prohermite_coeffmatrix(25)

  0.000016 seconds (97 allocations: 21.938 KiB)


26×26 Array{Float64,2}:
       1.0              0.0         …      0.0     0.0     0.0  0.0  0.0
       0.0              1.0                0.0     0.0     0.0  0.0  0.0
      -1.0              0.0                0.0     0.0     0.0  0.0  0.0
      -0.0             -3.0                0.0     0.0     0.0  0.0  0.0
       3.0             -0.0                0.0     0.0     0.0  0.0  0.0
       0.0             15.0         …      0.0     0.0     0.0  0.0  0.0
     -15.0              0.0                0.0     0.0     0.0  0.0  0.0
      -0.0           -105.0                0.0     0.0     0.0  0.0  0.0
     105.0             -0.0                0.0     0.0     0.0  0.0  0.0
       0.0            945.0                0.0     0.0     0.0  0.0  0.0
    -945.0              0.0         …      0.0     0.0     0.0  0.0  0.0
      -0.0         -10395.0                0.0     0.0     0.0  0.0  0.0
   10395.0             -0.0                0.0     0.0     0.0  0.0  0.0
       0.0         135135.0

In [30]:
Polynomial(Float64[])

Polynomial{1,Array{Float64,1}}((Float64[],))

In [11]:
@btime randn(5)

  48.958 ns (1 allocation: 128 bytes)


5-element Array{Float64,1}:
 -0.6116845457151265
  1.676031268501544
 -2.0524332537116585
  0.26267589888464804
 -0.2998856766106414

In [19]:
const ptest5 = Polynomial(Tuple(randn(5)))

Polynomial{5,Float64}((1.5425807952952248, -2.6169553675103763, 1.4518670230116697, 1.004786938649716, -1.2985790264559605))

In [21]:
@btime $ptest5(x) setup = x = rand()

  1.183 ns (0 allocations: 0 bytes)


0.45804629218533977

In [19]:
@btime ptest(4)

  0.015 ns (0 allocations: 0 bytes)


57.0

In [17]:
@time derivative(P)

  0.188276 seconds (863.38 k allocations: 39.300 MiB, 6.48% gc time)


Polynomial(8.7504519168e11*x^10)

In [18]:
phyhermite_coeffmatrix(12)

12×12 Array{Float64,2}:
 1.0       0.0   -2.0    -0.0     12.0  …  -30240.0        -0.0
 0.0       2.0    0.0     0.0      0.0          0.0         0.0
 0.0       0.0    4.0     0.0      0.0          0.0         0.0
 0.0     -12.0    0.0    56.0      0.0          0.0         0.0
 0.0      -0.0  -24.0     0.0    256.0          0.0         0.0
 0.0      24.0    0.0  -448.0      0.0  …       0.0         0.0
 0.0       0.0    0.0     0.0  -2560.0          0.0         0.0
 0.0    -240.0    0.0     0.0      0.0          0.0         0.0
 0.0      -0.0    0.0     0.0      0.0          0.0         0.0
 0.0    3360.0    0.0     0.0      0.0          0.0         0.0
 0.0       0.0    0.0     0.0      0.0  …       2.94027e9   0.0
 0.0  -60480.0    0.0     0.0      0.0          0.0         7.95496e10

In [19]:
@time P = ImmutablePolynomial([1,3,5, 0, 4])

  0.039477 seconds (177.27 k allocations: 8.458 MiB, 29.10% gc time)


ImmutablePolynomial(1 + 3*x + 5*x^2 + 4*x^4)

In [20]:
@btime P(1.0)

  11.946 ns (1 allocation: 16 bytes)


13.0

In [21]:
P

ImmutablePolynomial(1 + 3*x + 5*x^2 + 4*x^4)

In [22]:
fieldnames(typeof(P))

(:coeffs, :var)

In [23]:
P.var

:x

In [24]:
### Rescale variable 
ens = EnsembleState(randn(100,1000))
x̄ = mean(ens)
S = cov(ens) + 20*I
C = cholesky(S)
@time inv(C.L)

UndefVarError: UndefVarError: cov not defined

In [25]:
?ldiv!

search: ldiv! splitdrive



```
ldiv!(Y, A, B) -> Y
```

Compute `A \ B` in-place and store the result in `Y`, returning the result.

The argument `A` should *not* be a matrix.  Rather, instead of matrices it should be a factorization object (e.g. produced by [`factorize`](@ref) or [`cholesky`](@ref)). The reason for this is that factorization itself is both expensive and typically allocates memory (although it can also be done in-place via, e.g., [`lu!`](@ref)), and performance-critical situations requiring `ldiv!` usually also require fine-grained control over the factorization of `A`.

# Examples

```jldoctest
julia> A = [1 2.2 4; 3.1 0.2 3; 4 1 2];

julia> X = [1; 2.5; 3];

julia> Y = zero(X);

julia> ldiv!(Y, qr(A), X);

julia> Y
3-element Array{Float64,1}:
  0.7128099173553719
 -0.051652892561983674
  0.10020661157024757

julia> A\X
3-element Array{Float64,1}:
  0.7128099173553719
 -0.05165289256198333
  0.10020661157024785
```

---

```
ldiv!(A, B)
```

Compute `A \ B` in-place and overwriting `B` to store the result.

The argument `A` should *not* be a matrix.  Rather, instead of matrices it should be a factorization object (e.g. produced by [`factorize`](@ref) or [`cholesky`](@ref)). The reason for this is that factorization itself is both expensive and typically allocates memory (although it can also be done in-place via, e.g., [`lu!`](@ref)), and performance-critical situations requiring `ldiv!` usually also require fine-grained control over the factorization of `A`.

# Examples

```jldoctest
julia> A = [1 2.2 4; 3.1 0.2 3; 4 1 2];

julia> X = [1; 2.5; 3];

julia> Y = copy(X);

julia> ldiv!(qr(A), X);

julia> X
3-element Array{Float64,1}:
  0.7128099173553719
 -0.051652892561983674
  0.10020661157024757

julia> A\Y
3-element Array{Float64,1}:
  0.7128099173553719
 -0.05165289256198333
  0.10020661157024785
```

---

```
ldiv!(a::Number, B::AbstractArray)
```

Divide each entry in an array `B` by a scalar `a` overwriting `B` in-place.  Use [`rdiv!`](@ref) to divide scalar from right.

# Examples

```jldoctest
julia> B = [1.0 2.0; 3.0 4.0]
2×2 Array{Float64,2}:
 1.0  2.0
 3.0  4.0

julia> ldiv!(2.0, B)
2×2 Array{Float64,2}:
 0.5  1.0
 1.5  2.0
```

---

Solve Hy = rhs for a non-square Hessenberg matrix. Note that `H` is also modified as is it converted to an upper triangular matrix via Given's rotations


In [26]:
?cholesky

search: cholesky cholesky! Cholesky CholeskyPivoted



```
cholesky(A, Val(false); check = true) -> Cholesky
```

Compute the Cholesky factorization of a dense symmetric positive definite matrix `A` and return a `Cholesky` factorization. The matrix `A` can either be a [`Symmetric`](@ref) or [`Hermitian`](@ref) `StridedMatrix` or a *perfectly* symmetric or Hermitian `StridedMatrix`. The triangular Cholesky factor can be obtained from the factorization `F` with: `F.L` and `F.U`. The following functions are available for `Cholesky` objects: [`size`](@ref), [`\`](@ref), [`inv`](@ref), [`det`](@ref), [`logdet`](@ref) and [`isposdef`](@ref).

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

# Examples

```jldoctest
julia> A = [4. 12. -16.; 12. 37. -43.; -16. -43. 98.]
3×3 Array{Float64,2}:
   4.0   12.0  -16.0
  12.0   37.0  -43.0
 -16.0  -43.0   98.0

julia> C = cholesky(A)
Cholesky{Float64,Array{Float64,2}}
U factor:
3×3 UpperTriangular{Float64,Array{Float64,2}}:
 2.0  6.0  -8.0
  ⋅   1.0   5.0
  ⋅    ⋅    3.0

julia> C.U
3×3 UpperTriangular{Float64,Array{Float64,2}}:
 2.0  6.0  -8.0
  ⋅   1.0   5.0
  ⋅    ⋅    3.0

julia> C.L
3×3 LowerTriangular{Float64,Array{Float64,2}}:
  2.0   ⋅    ⋅
  6.0  1.0   ⋅
 -8.0  5.0  3.0

julia> C.L * C.U == A
true
```

---

```
cholesky(A, Val(true); tol = 0.0, check = true) -> CholeskyPivoted
```

Compute the pivoted Cholesky factorization of a dense symmetric positive semi-definite matrix `A` and return a `CholeskyPivoted` factorization. The matrix `A` can either be a [`Symmetric`](@ref) or [`Hermitian`](@ref) `StridedMatrix` or a *perfectly* symmetric or Hermitian `StridedMatrix`. The triangular Cholesky factor can be obtained from the factorization `F` with: `F.L` and `F.U`. The following functions are available for `CholeskyPivoted` objects: [`size`](@ref), [`\`](@ref), [`inv`](@ref), [`det`](@ref), and [`rank`](@ref). The argument `tol` determines the tolerance for determining the rank. For negative values, the tolerance is the machine precision.

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

---

```
cholesky(A; shift = 0.0, check = true, perm = nothing) -> CHOLMOD.Factor
```

Compute the Cholesky factorization of a sparse positive definite matrix `A`. `A` must be a [`SparseMatrixCSC`](@ref) or a [`Symmetric`](@ref)/[`Hermitian`](@ref) view of a `SparseMatrixCSC`. Note that even if `A` doesn't have the type tag, it must still be symmetric or Hermitian. If `perm` is not given, a fill-reducing permutation is used. `F = cholesky(A)` is most frequently used to solve systems of equations with `F\b`, but also the methods [`diag`](@ref), [`det`](@ref), and [`logdet`](@ref) are defined for `F`. You can also extract individual factors from `F`, using `F.L`. However, since pivoting is on by default, the factorization is internally represented as `A == P'*L*L'*P` with a permutation matrix `P`; using just `L` without accounting for `P` will give incorrect answers. To include the effects of permutation, it's typically preferable to extract "combined" factors like `PtL = F.PtL` (the equivalent of `P'*L`) and `LtP = F.UP` (the equivalent of `L'*P`).

When `check = true`, an error is thrown if the decomposition fails. When `check = false`, responsibility for checking the decomposition's validity (via [`issuccess`](@ref)) lies with the user.

Setting the optional `shift` keyword argument computes the factorization of `A+shift*I` instead of `A`. If the `perm` argument is provided, it should be a permutation of `1:size(A,1)` giving the ordering to use (instead of CHOLMOD's default AMD ordering).

# Examples

In the following example, the fill-reducing permutation used is `[3, 2, 1]`. If `perm` is set to `1:3` to enforce no permutation, the number of nonzero elements in the factor is 6.

```jldoctest
julia> A = [2 1 1; 1 2 0; 1 0 2]
3×3 Array{Int64,2}:
 2  1  1
 1  2  0
 1  0  2

julia> C = cholesky(sparse(A))
SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  simplicial
maxnnz:  5
nnz:     5
success: true

julia> C.p
3-element Array{Int64,1}:
 3
 2
 1

julia> L = sparse(C.L);

julia> Matrix(L)
3×3 Array{Float64,2}:
 1.41421   0.0       0.0
 0.0       1.41421   0.0
 0.707107  0.707107  1.0

julia> L * L' ≈ A[C.p, C.p]
true

julia> P = sparse(1:3, C.p, ones(3))
3×3 SparseMatrixCSC{Float64,Int64} with 3 stored entries:
  [3, 1]  =  1.0
  [2, 2]  =  1.0
  [1, 3]  =  1.0

julia> P' * L * L' * P ≈ A
true

julia> C = cholesky(sparse(A), perm=1:3)
SuiteSparse.CHOLMOD.Factor{Float64}
type:    LLt
method:  simplicial
maxnnz:  6
nnz:     6
success: true

julia> L = sparse(C.L);

julia> Matrix(L)
3×3 Array{Float64,2}:
 1.41421    0.0       0.0
 0.707107   1.22474   0.0
 0.707107  -0.408248  1.1547

julia> L * L' ≈ A
true
```

!!! note
    This method uses the CHOLMOD library from SuiteSparse, which only supports doubles or complex doubles. Input matrices not of those element types will be converted to `SparseMatrixCSC{Float64}` or `SparseMatrixCSC{ComplexF64}` as appropriate.

    Many other functions from CHOLMOD are wrapped but not exported from the `Base.SparseArrays.CHOLMOD` module.

